# diatom_scan - Python class

__Python imports__

- [numpy](http://www.numpy.org/)
- [IPython](https://ipython.org)
- [atomman](https://github.com/usnistgov/atomman)
- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from pathlib import Path
import datetime
from math import floor

# http://www.numpy.org/
import numpy as np

# https://ipython.org/
from IPython.display import display, Markdown

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

print('Notebook last executed on', datetime.date.today(), 'using iprPy version', iprPy.__version__)

Notebook last executed on 2022-03-03 using iprPy version 0.11.2


Import additional libraries for plotting. The external libraries used are:

- [bokeh](http://bokeh.pydata.org/)

In [2]:
import bokeh
print('Bokeh version =', bokeh.__version__)
from bokeh.plotting import figure, output_file, show
from bokeh.embed import components
from bokeh.resources import Resources
from bokeh.io import output_notebook
output_notebook()

Bokeh version = 2.4.2


Loading BokehJS ...

## 1. Load calculation and view description

### 1.1. Load the calculation

In [3]:
# Load the calculation being demoed
calculation = iprPy.load_calculation('diatom_scan')

### 1.2. Display calculation description and theory

In [4]:
# Display main docs and theory
display(Markdown(calculation.maindoc))
display(Markdown(calculation.theorydoc))

# diatom_scan calculation style

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

## Introduction

The diatom_scan calculation style evaluates the interaction energy between two atoms at varying distances.  This provides a measure of the isolated pair interaction of two atoms providing insights into the strengths of the attraction/repulsion and the effective range of interatomic spacings.  This scan also gives insight into the computational smoothness of the potential's functional form.

### Version notes

- 2019-07-30: Notebook added.
- 2020-05-22: Version 0.10 update - potentials now loaded from database.
- 2020-09-22: Setup and parameter definition streamlined. Method and theory expanded.
- 2022-02-16: Notebook updated to reflect version 0.11.

### Additional dependencies

### Disclaimers

- [NIST disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)
- No 3+ body interactions are explored with this calculation as only two atoms are used.


## Method and Theory

Two atoms are placed in an otherwise empty system.  The total energy of the system is evaluated for different interatomic spacings.  This provides a means of evaluating the pair interaction component of an interatomic potential, which is useful for a variety of reasons

- The diatom_scan is a simple calculation that can be used to fingerprint a given interaction.  This can be used to help determine if two different implementations produce the same resulting potential when direct comparisons of the potential parameters is not feasible.
- For a potential to be suitable for radiation studies, the extreme close-range interaction energies must be prohibitively repulsive while not being so large that the resulting force on the atoms will eject them from the system during integration.  The diatom_scan results provide a means of evaluating the close-range interactions.
- The smoothness of the potential is also reflected in the diatom_scan energy results.  Numerical derivatives of the measured points can determine the order of smoothness as well as the approximate r values where discontinuities occur.
- Evaluating large separation values provides a means of identifying the energy of the isolated atoms, given that the separation exceeds the potential's cutoff.  The isolated_atom calculation is an alternative method for obtaining this.


## 2. Specify input parameters

The calculation input parameters can be specified directly to the associated calculation's attributes.  Note that these parameters are often part of calculation subsets allowing multiple calculations to share the same input terms.

### 2.1. LAMMPS-related commands

Calculations that use LAMMPS need to know which LAMMPS executable to use.  Additionally, running LAMMPS in parallel requires specifying an MPI command.

- __commands__ is a LammpsCommands subset.
    - __lammps_command__ is the LAMMPS command to use.
    - __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

In [5]:
calculation.commands.lammps_command = 'lmp_serial'
calculation.commands.mpi_command = None

### 2.2. Interatomic potential

LAMMPS calculations also need to know which potential to use.  These are handled by loaded potentials/atomman PotentialLAMMPS objects.

- __potential__ is a LammpsPotential subset.
    - __potential__ is a loaded PotentialLAMMPS or PotentialLAMMPSKIM object.

In [6]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
potential = am.load_lammps_potential(id=potential_name, getfiles=True)

# So many "potential"s...
calculation.potential.potential = potential

### 2.3. Calculation-specific parameters

- __symbols__ is the element or pair of element model symbols to use for the diatom.
- __rmin__ is the minimum r spacing to use.
- __rmax__ is the minimum r spacing to use.
- __rsteps__ is the number of r spacing steps to evaluate.

In [7]:
calculation.symbols = 'Ni'
calculation.rmin = uc.set_in_units(0.02, 'angstrom')
calculation.rmax = uc.set_in_units(6.0, 'angstrom')
calculation.rsteps = 300

## 3. Preview the data record

Optionally, an incomplete JSON/XML record can be generated for the calculation after setting the inputs.

- __build_model()__ will interpret the object's attributes into a data model.
- __model__ is the generated data model.
- __model.json()__ will convert the model to JSON.
- __model.xml()__ will convert the model to XML.

In [8]:
calculation.build_model()
print(calculation.model.json(indent=4))

{
    "calculation-diatom-scan": {
        "key": "5a58f5c4-b641-4cf9-9f4a-b976d6b13b65",
        "calculation": {
            "iprPy-version": "0.11.2",
            "atomman-version": "1.4.3",
            "LAMMPS-version": "3 Mar 2020",
            "script": "calc_diatom_scan",
            "branch": "main",
            "run-parameter": {
                "minimum_r": {
                    "value": 0.02,
                    "unit": "angstrom"
                },
                "maximum_r": {
                    "value": 6.0,
                    "unit": "angstrom"
                },
                "number_of_steps_r": 300
            }
        },
        "potential-LAMMPS": {
            "key": "a7c9b786-5aa7-481b-86eb-7e4edd9cec02",
            "id": "1999--Mishin-Y--Ni--LAMMPS--ipr1",
            "potential": {
                "key": "81adb388-59eb-4f49-88bf-6f06a1343fae",
                "id": "1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Ni"
            }
        },
   

## 4. Run calculation and view results

### 4.1. Run calculation
 
__run()__ Runs the calculation using the current object attribute values or supplied parameters. Status after running will be either "finished" or "error".
        
- __params__ (*dict, str or file-like object, optional*) The key-value parameter file or dict of associated values to read in.  If not given, will run based on the current object attribute values.
- __newkey__ (*bool, optional*) If True, then the calculation's key and name will be replaced with a new UUID4.  This allows for iterations on previous runs to be uniquely labeled.  Default value is False.
- __results_json__ (*bool, optional*) If True, then a "results.json" file will be generated following the run.
- __verbose__ (*bool, optional*) If True, a message relating to the calculation's status will be printed upon completion.  Default value is False.

In [9]:
calculation.run(verbose=True)

Calculation finished successfully


### 4.2. Results attributes

- __isolated_atom_energy__ (*dict*) contains the computed isolated atom energies for each of the potential's symbol models.  

In [10]:
length_unit = 'angstrom'
energy_unit = 'eV'

energy = uc.get_in_units(calculation.energy_values, energy_unit)
r = uc.get_in_units(calculation.r_values, length_unit)

Emin = floor(energy.min())
if Emin < -10: 
    Emin = -10
    
plot = figure(title = f'Diatom energy scan for {calculation.potential.potential_LAMMPS_id}',
              plot_width = 800,
              plot_height = 600,
              x_range = [uc.get_in_units(calculation.rmin, 'angstrom'),
                         uc.get_in_units(calculation.rmax, 'angstrom')],
              y_range = [Emin, 0],              
              x_axis_label=f'r ({length_unit})', 
              y_axis_label=f'Cohesive Energy ({energy_unit}/atom)')

plot.line(r, energy, line_width = 2, legend_label = '-'.join(calculation.symbols))            
plot.legend.location = "bottom_right"    

show(plot)

### 4.3. Output as JSON/XML

In [11]:
calculation.build_model()
print(calculation.model.json(indent=4))

{
    "calculation-diatom-scan": {
        "key": "5a58f5c4-b641-4cf9-9f4a-b976d6b13b65",
        "calculation": {
            "iprPy-version": "0.11.2",
            "atomman-version": "1.4.3",
            "LAMMPS-version": "3 Mar 2020",
            "script": "calc_diatom_scan",
            "branch": "main",
            "run-parameter": {
                "minimum_r": {
                    "value": 0.02,
                    "unit": "angstrom"
                },
                "maximum_r": {
                    "value": 6.0,
                    "unit": "angstrom"
                },
                "number_of_steps_r": 300
            }
        },
        "potential-LAMMPS": {
            "key": "a7c9b786-5aa7-481b-86eb-7e4edd9cec02",
            "id": "1999--Mishin-Y--Ni--LAMMPS--ipr1",
            "potential": {
                "key": "81adb388-59eb-4f49-88bf-6f06a1343fae",
                "id": "1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstantopoulos-D-A--Ni"
            }
        },
   